In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv as csv
%matplotlib inline

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
train2= pd.read_csv("train.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
test2= pd.read_csv("test.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)

In [8]:
test2["Fare"].fillna(test.Fare.median(), inplace=True)

In [9]:
train2["Family_size"] = train2["SibSp"] + train2["Parch"] +1 
test2["Family_size"] = test2["SibSp"] + test2["Parch"] +1

In [10]:
train2["Alone"] = 0
train2.loc[train2["Family_size"] == 1, "Alone"] = 1
test2["Alone"] = 0
test2.loc[test2["Family_size"] == 1, "Alone"] = 1

In [11]:
for train2 in [train2]:
        train2['Title'] = train2.Name.str.extract(' ([A-Za-z]+).', expand=False)
for train2 in [train2]:
        train2['Title'] = train2['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
        train2['Title'] = train2['Title'].replace('Mlle', 'Miss')
        train2['Title'] = train2['Title'].replace('Ms', 'Miss')
        train2['Title'] = train2['Title'].replace('Mme', 'Mrs')
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Other": 5} 

for train2 in [train2]: 
        train2['Title'] = train2['Title'].map(Salutation_mapping) 
        train2['Title'] = train2['Title'].fillna(0)

In [12]:
for test2 in [test2]:
        test2['Title'] = test2.Name.str.extract('([A-Za-z]+).', expand=False) 
for test2 in [test2]:
        test2['Title'] = test2['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
        test2['Title'] = test2['Title'].replace('Mlle', 'Miss')
        test2['Title'] = test2['Title'].replace('Ms', 'Miss')
        test2['Title'] = test2['Title'].replace('Mme', 'Mrs')
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Other": 5}

for test2 in [test2]:
        test2['Title'] = test2['Title'].map(Salutation_mapping)
        test2['Title'] = test2['Title'].fillna(0)

In [13]:
train2.loc[(train2['Name'].str.contains('Mr\.')) & (train2['Age'].isnull()), 'Age'] = train2[train2['Name'].str.contains('Mr\.')].Age.mean()
train2.loc[(train2['Name'].str.contains('Mrs\.')) & (train2['Age'].isnull()), 'Age'] = train2[train2['Name'].str.contains('Mrs\.')].Age.mean()
train2.loc[(train2['Name'].str.contains('Miss\.')) & (train2['Age'].isnull()), 'Age'] = train2[train2['Name'].str.contains('Miss\.')].Age.mean()
train2.loc[(train2['Name'].str.contains('Master\.')) & (train2['Age'].isnull()), 'Age'] = train2[train2['Name'].str.contains('Master\.')].Age.mean()
train2.loc[(train2['Name'].str.contains('Other\.')) & (train2['Age'].isnull()), 'Age'] = train2[train2['Name'].str.contains('Other\.')].Age.mean()

In [14]:
test2.loc[(test2['Name'].str.contains('Mr\.')) & (test2['Age'].isnull()), 'Age'] = test2[test2['Name'].str.contains('Mr\.')].Age.mean()
test2.loc[(test2['Name'].str.contains('Mrs\.')) & (test2['Age'].isnull()), 'Age'] = test2[test2['Name'].str.contains('Mrs\.')].Age.mean()
test2.loc[(test2['Name'].str.contains('Miss\.')) & (test2['Age'].isnull()), 'Age'] = test2[test2['Name'].str.contains('Miss\.')].Age.mean()
test2.loc[(test2['Name'].str.contains('Master\.')) & (test2['Age'].isnull()), 'Age'] = test2[test2['Name'].str.contains('Master\.')].Age.mean()
test2.loc[(test2['Name'].str.contains('Other\.')) & (test2['Age'].isnull()), 'Age'] = test2[test2['Name'].str.contains('Other\.')].Age.mean()

In [16]:
import xgboost as xgb
from xgboost import XGBClassifier

target = train2["Survived"].values
features = train2[["Pclass","Age","Sex","Fare","Family_size","Alone"]].values
model = XGBClassifier(n_estimators=30, random_state=9)
model = model.fit(features, target)

test_features = test2[["Pclass","Age","Sex","Fare","Family_size","Alone"]].values
prediction = model.predict(test_features)
PassengerId = np.array(test["PassengerId"]).astype(int)
output = pd.DataFrame(prediction, PassengerId, columns=["Survived"])
output.to_csv("submit.csv", index_label=["PassengerId"])